In [146]:
import re
from pyvi import ViTokenizer
import pandas as pd

class DataPreprocessing:
    def __init__(self, text, acronyms_file, stopwords_file):
        self.text = text
        self.stopwords = self.load_stopwords(stopwords_file)
        self.acronyms = self.load_acronyms(acronyms_file)

    def load_acronyms(self, filename):
        """Tải danh sách từ viết tắt từ file."""
        acronyms = []
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                for line in f:
                    parts = line.strip().split(maxsplit=1)  # Tách bằng khoảng trắng đầu tiên
                    if len(parts) == 2:
                        acronyms.append((parts[0].strip(), parts[1].strip()))
        except FileNotFoundError:
            print(f"File {filename} không tồn tại.")
        return acronyms

    def load_stopwords(self, filename):
        """Tải danh sách từ dừng từ file."""
        stopwords = set()
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                for line in f:
                    stopwords.add(line.strip())
        except FileNotFoundError:
            print(f"File {filename} không tồn tại.")
        return stopwords
    
    def remove_punctuation(self, text):
        return re.sub(r'[^\w\s]', '', text)

    def segmentation(self):
        return ViTokenizer.tokenize(self.text)

    def split_words(self):
        text = self.segmentation()
        try:
            special_characters = '.,!?;:"\'()[]{}<>-'
            cleaned_words = [x.strip(special_characters).lower() for x in text.split()]
            return [self.remove_punctuation(word) for word in cleaned_words]
        except TypeError:
            return []

    def remove_loop_char(self):
        self.text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), str(self.text), flags=re.IGNORECASE)
        self.text = re.sub(r'[^A-Za-zÀ-ỹ\s]', ' ', self.text)
        return self.text
        
    def replace_acronyms(self):
        """Thay thế các từ viết tắt."""
        list_text = self.text.split(" ")
        for i in range(len(list_text)):
            for acronym in self.acronyms:
                if list_text[i] == acronym[0]:
                    list_text[i] = acronym[1]
        self.text = " ".join(list_text)
        return self.text

    def remove_stopwords(self):
        split_words = self.split_words()
        words = []

        for word in split_words:
            if word not in self.stopwords:
                words.append(word)

        return ' '.join(words)

    def preprocess(self):
        self.text = self.remove_loop_char()
        self.text = self.replace_acronyms()
        self.text = self.remove_stopwords()
        return self.text

# Đường dẫn tới tệp CSV
csv_file = './datasets.csv'  # Thay đổi tên tệp này theo tệp của bạn
acronyms_file = './acronym_vi.txt'  # Đường dẫn tới tệp từ viết tắt
stopwords_file = './stopwords-nlp-vi.txt'  # Đường dẫn tới tệp từ dừng

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv(csv_file)

# Chuẩn hóa nội dung
df['content'] = df['content'].apply(lambda x: DataPreprocessing(x, acronyms_file, stopwords_file).preprocess())
df['category'] = df['category']  # Giữ nguyên cột category 

# Lưu kết quả vào tệp CSV mới
df[['content', 'category']].to_csv('output_processed.csv', index=False, encoding='utf-8-sig')

In [144]:
# In thông tin của csv đã được tiền xử lý dữ liệu
output_csv_file = 'output_processed.csv' 

# Đọc dữ liệu từ tệp CSV
df_processed = pd.read_csv(output_csv_file)

# In thông tin từ DataFrame
print(df_processed.info())  
print(df_processed.head())   
print(df_processed.describe())   


unique_categories = df_processed['category'].unique()
print(unique_categories)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9504 entries, 0 to 9503
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   content   9504 non-null   object
 1   category  9504 non-null   object
dtypes: object(2)
memory usage: 148.6+ KB
None
                                             content    category
0  cá_nhân_hóa trải nghiệm công_nghệ tiên_tiến ng...  Apps/Games
1  microsoft ra_mắt ofice ofice chính_thức ra_mắt...  Apps/Games
2  thành_tựu black myth wukong youtube ải như_khô...  Apps/Games
3  tựa trò_chơi đắt steam mua nội_dung tốn triệu ...  Apps/Games
4  ceo trò_chơi science trả_lời hài_hước khả_năng...  Apps/Games
                                                  content category
count                                                9504     9504
unique                                               9503        5
top     tựa trò_chơi miễn_phí đồng cực chất steam thời...  Tin ICT
freq                                 

In [151]:
# =====================================
# =====================================
# Vector hóa văn bản
# =====================================
# =====================================
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

csv_file_path = './output_processed.csv'

df = pd.read_csv(csv_file_path)

# Đếm số lượng bản ghi cho mỗi lớp trong cột 'category'
record_counts = df['category'].value_counts()
print(record_counts)

X = df['content']
y = df['category']

# Vector hóa nội dung
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Chuyển đổi nhãn lớp thành số
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Kiểm tra kích thước
print(f"Kích thước X_train: {X_train.shape}")
print(f"Kích thước X_test: {X_test.shape}")
print(f"Số lớp trong y_train: {len(set(y_train))}")

category
Tin ICT       2064
Mobile        1968
Internet      1910
Đồ chơi số    1886
Apps/Games    1676
Name: count, dtype: int64
Kích thước X_train: (7603, 14292)
Kích thước X_test: (1901, 14292)
Số lớp trong y_train: 5


LOGISTIC REGRESSION

In [157]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def train_logistic_regression(X_train, y_train, X_test, y_test):
    # Khởi tạo mô hình Logistic Regression
    model = LogisticRegression(max_iter=1000)

    # Huấn luyện mô hình
    model.fit(X_train, y_train)

    # Dự đoán trên tập kiểm tra
    y_pred = model.predict(X_test)

    # Tính toán các chỉ số đánh giá
    acc = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    # Chỉ số F1, Recall, Precision cho lớp chính
    f1 = report['macro avg']['f1-score']
    recall = report['macro avg']['recall']
    precision = report['macro avg']['precision']
    
    # In ra kết quả
    print("Model: Logistic Regression")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)
    print("\n" + "="*50 + "\n")

# Ví dụ cách gọi hàm với dữ liệu
train_logistic_regression(X_train, y_train, X_test, y_test)

Model: Logistic Regression
Accuracy: 0.7070
F1 Score: 0.7093
Recall: 0.7076
Precision: 0.7114
Confusion Matrix:
[[261  37  14  27   7]
 [ 21 236   7 100  19]
 [ 19   8 329  22  24]
 [ 34  96  20 220  21]
 [ 10  20  14  37 298]]




RANDOM FOREST

In [158]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def train_random_forest(X_train, y_train, X_test, y_test):
    # Khởi tạo mô hình Random Forest
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Huấn luyện mô hình
    model.fit(X_train, y_train)

    # Dự đoán trên tập kiểm tra
    y_pred = model.predict(X_test)

    # Tính toán các chỉ số đánh giá
    acc = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    # Chỉ số F1, Recall, Precision cho lớp chính
    f1 = report['macro avg']['f1-score']
    recall = report['macro avg']['recall']
    precision = report['macro avg']['precision']
    
    # In ra kết quả
    print("Model: Random Forest")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)
    print("\n" + "="*50 + "\n")

# Ví dụ cách gọi hàm với dữ liệu
train_random_forest(X_train, y_train, X_test, y_test)


Model: Random Forest
Accuracy: 0.6828
F1 Score: 0.6786
Recall: 0.6832
Precision: 0.6775
Confusion Matrix:
[[255  40  24  17  10]
 [ 27 235   9  82  30]
 [ 24   3 344  12  19]
 [ 40 101  32 180  38]
 [ 16  28  30  21 284]]




SVM

In [160]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def train_svm(X_train, y_train, X_test, y_test):
    # Khởi tạo mô hình SVM
    model = SVC(kernel='linear', random_state=42)

    # Huấn luyện mô hình
    model.fit(X_train, y_train)

    # Dự đoán trên tập kiểm tra
    y_pred = model.predict(X_test)

    # Tính toán các chỉ số đánh giá
    acc = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    # Chỉ số F1, Recall, Precision cho lớp chính
    f1 = report['macro avg']['f1-score']
    recall = report['macro avg']['recall']
    precision = report['macro avg']['precision']
    
    # In ra kết quả
    print("Model: SVM")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)
    print("\n" + "="*50 + "\n")

    return model

# Ví dụ cách gọi hàm với dữ liệu
train_svm(X_train, y_train, X_test, y_test)

Model: SVM
Accuracy: 0.7065
F1 Score: 0.7079
Recall: 0.7073
Precision: 0.7087
Confusion Matrix:
[[268  35  15  22   6]
 [ 24 229   7 104  19]
 [ 15   8 336  22  21]
 [ 36  90  22 218  25]
 [ 10  19  21  37 292]]




SVC(kernel='linear', random_state=42)

Predict

In [163]:
def predict_svm(model, vectorizer, text):
    # Chuyển đổi văn bản thành vector bằng vectorizer đã được huấn luyện
    text_vector = vectorizer.transform([text])

    # Dự đoán lớp cho văn bản
    prediction = model.predict(text_vector)

    return prediction[0]

trained_model = train_svm(X_train, y_train, X_test, y_test)

Model: SVM
Accuracy: 0.7065
F1 Score: 0.7079
Recall: 0.7073
Precision: 0.7087
Confusion Matrix:
[[268  35  15  22   6]
 [ 24 229   7 104  19]
 [ 15   8 336  22  21]
 [ 36  90  22 218  25]
 [ 10  19  21  37 292]]




In [168]:
classes = ['Apps/Games', 'Internet', 'Mobile', 'Tin ICT', 'Đồ chơi số']
text_input = "AI khôi phục chân dung các nhân vật nổi tiếng Trung Quốc gây ngỡ ngàng: Tào Tháo, Tần Thuỷ Hoàng khác xa tưởng tượng, Trương Phi khiến người nhìn hoảng sợ"
prediction = predict_svm(trained_model, vectorizer, text_input)
print(f"Dự đoán lớp: { classes[prediction] }")

Dự đoán lớp: Internet


In [145]:
# =====================================
# =====================================
# Biến đổi chuỗi json thành csv
# =====================================
# =====================================
import pandas as pd
import json
import csv

def json_to_csv(json_file, csv_file):
    # Đọc dữ liệu từ tệp JSON
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Chuyển đổi dữ liệu JSON thành DataFrame
    df = pd.json_normalize(data)

    # Tạo trường mới là nối title và subtitle
    df['content'] = df['title'] + " " + df['subtitle']

   # Chọn các cột cần thiết
    result_df = df[['content', 'category']]

    # Lọc các hàng có category thuộc danh sách mong muốn
    categories_to_keep = ['Apps/Games', 'Internet', 'Mobile', 'Tin ICT', 'Đồ chơi số']
    filtered_df = result_df[result_df['category'].isin(categories_to_keep)]

    # Loại bỏ các dòng trùng lặp dựa trên cột 'content'
    filtered_df = filtered_df.drop_duplicates(subset='content')

    # Ghi DataFrame vào tệp CSV
    filtered_df.to_csv(csv_file, index=False, encoding='utf-8-sig')


# Đường dẫn tới tệp JSON và tệp CSV mong muốn
json_file_path = './datasets.json'  # Thay thế bằng đường dẫn tệp JSON của bạn
csv_file_path = './datasets.csv'     # Thay thế bằng đường dẫn tệp CSV bạn muốn tạo

# Gọi hàm chuyển đổi
json_to_csv(json_file_path, csv_file_path)

print(f"Đã chuyển đổi {json_file_path} thành {csv_file_path}")


Đã chuyển đổi ./datasets.json thành ./datasets.csv
